In [1]:
import numpy as np
import imageio as iio
import DatasetSplitter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA as pca_test


# Splitting images to training data and testing data

In [2]:
trainingData, trainingLabels, testingData, testingLabels = DatasetSplitter.splitData()

# Computing covariance matrix

In [3]:
def covariance(D):
    global mean_vector 
    mean_vector = np.mean(D, axis=0)
    Z = D - mean_vector
    cov = (1/len(D)) * (Z.T @ Z) 
    return cov


In [ ]:
# try
# def covariance(D):
#     global mean_vector 
#     mean_vector = np.mean(D, axis=0)
#     global Z 
#     Z = D - mean_vector
#     cov = (1/len(D))*(Z @ Z.T) 
#     return cov

# Testing computational correctness of covariance matrix

In [4]:
# Test cov matrix 
trainingData = np.array(trainingData)
myCov = covariance(trainingData)
numCov = np.cov(trainingData.T, bias=True)
# print(numCov)

# Applying PCA algorithm to obtain P -> projection matrix

In [6]:
def PCA(D, alpha):
    cov = covariance(D)
    eigenvalues, eigenvectors = np.linalg.eigh(cov)
    
    sorted_idx = eigenvalues.argsort()[::-1]  # Sort in descending order
    eigenvalues = eigenvalues[sorted_idx]
    eigenvectors = eigenvectors[:, sorted_idx]
    
    trace = np.trace(cov)
    c = 0 # Accumulator for sum of eigenvaleus
    lastIndex = 0
    for index in range(len(eigenvalues)):
        c += eigenvalues[index]
        if c / trace >= alpha :
            lastIndex = index
            break
    P = eigenvectors[:lastIndex + 1, :]# Projection Matrix
    return P

In [ ]:
# # try
# def PCA(D, alpha):
#     cov = covariance(D)
#     eigenvalues, eigenvectors = np.linalg.eigh(cov)
    
#     sorted_idx = eigenvalues.argsort()[::-1]  # Sort in descending order
#     eigenvalues = eigenvalues[sorted_idx]
#     eigenvectors = eigenvectors[:, sorted_idx]
    
#     eigenvectors = Z.T @ eigenvectors
    
#     eigenvectors = eigenvectors.T/ np.sqrt((eigenvectors.T ** 2).sum(axis=1, keepdims=True))
    
#     trace = sum(eigenvalues)
#     c = 0 # Accumulator for sum of eigenvaleus
#     lastIndex = 0
#     for index in range(len(eigenvalues)):
#         c += eigenvalues[index]
#         if c / trace >= alpha :
#             lastIndex = index
#             break
#     P = eigenvectors[:lastIndex + 1, :]# Projection Matrix
#     return P

In [7]:
def projectData(trainData, testData, alpha):
    P = PCA(trainData, alpha)
    # Project training data (correctly subtract mean before projection)
    projectedTrainingData = (trainData - mean_vector) @ P.T

    # Project testing data (correctly subtract mean before projection)
    projectedTestingData = (testData - mean_vector) @ P.T
    
    return projectedTrainingData, projectedTestingData

# Using KNN classifier, tie breaking at distance strategy

In [8]:
def classifyKNN(projectedTrainData, trainLabel, projectedTestData, testLabel, k):
    knn = KNeighborsClassifier(n_neighbors=k, weights='distance')
    knn.fit(projectedTrainData, trainLabel)
    # Calc accuracy of clssification
    predictedClasses = knn.predict(projectedTestData)
    accuracy = accuracy_score(testLabel, predictedClasses)
    print(f"Accuracy {100 * (accuracy)}")
    return accuracy

# Main Flow

In [9]:
alpha_list = np.array([0.8, 0.85, 0.9, 0.95])
accuracy_results = np.array([])
k = 1
for alpha in alpha_list:
    # Train the model
    projectedTrainingData, projectedTestingData = projectData(trainingData, testingData, alpha)
    # Classify and test
    accuracy_results = np.append(accuracy_results, classifyKNN(
        projectedTrainingData, trainingLabels, 
        projectedTestingData, testingLabels,
        k
    ))
      

Accuracy 68.5
Accuracy 81.0


# Plot the results

In [ ]:
# plotting the points 
plt.plot(alpha_list, accuracy_results)
plt.xlabel('alpha')
plt.ylabel('accuracy')
plt.title('PCA - Part 4')
plt.show()  

According to this plot, accuracy is proportional to alpha. 

# Hyper-parameter tuning

In [ ]:
k_list = np.array([1, 3, 5, 7])
accuracy_results = np.array([])
alpha = 0.95
for k in k_list:
    # Train the model
    projectedTrainingData, projectedTestingData = projectData(trainingData, testingData, alpha)
    # Classify and test
    accuracy_results = np.append(accuracy_results, classifyKNN(
        projectedTrainingData, trainingLabels, 
        projectedTestingData, testingLabels,
        k
    ))

In [ ]:
# plotting the points 
plt.plot(k_list, accuracy_results)
plt.xlabel('K (n_neigbour)')
plt.ylabel('accuracy')
plt.title('PCA - Part 6')
plt.show()  

In our case the best K is K = 1 

# Bonus

# Using randomized PCA for fast computation 

The time complexity of randomized PCA is O(n * p^2) + O(p^3) where p is the number of PCs. Meanwhile, The time complexity of traditional PCA is O(n * d^2) + O(d^3)

In [ ]:
k = 1
numPca = pca_test(n_components=114, svd_solver='randomized')
transformedTrainingData = pca.fit_transform(trainingData)
transformedTestingData = pca.transform(testingData)
accuracy = classifyKNN(
        transformedTrainingData, trainingLabels, 
        transformedTestingData, testingLabels,
        k
    )
print(f"Randomized PCA at K = 1")


The accuracy is better than mine but In general it's less than the traditional PCA.

The reason why our PCA is less better tha pyhon's one is the propagation error (truncation error) at calculating the covariance matrix